In [52]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [53]:

mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

cap = cv2.VideoCapture(0)


In [54]:
# pose= []
# for res in results.pose_landmarks.landmark:
#     test = np.array([res.x,res.y,res.z,res.visibility])
#     pose.append(test)
cap.release()
cv2.destroyAllWindows()

In [55]:
# pose = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten()

In [56]:
# if results.left_hand_landmarks: 
#     lh = np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten()
# else :
#     lh = np.zeros(21*3)
# if results.right_hand_landmarks: 
#     rh = np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten()
# else :
#     rh = np.zeros(21*3)
# if results.face_landmarks: 
#     face = np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten()
# else :
#     rh = np.zeros(21*3)

In [57]:
def extract_keypoints(results):
    if results.left_hand_landmarks: 
        pose = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten()
    else :
        pose = np.zeros(132)
    if results.left_hand_landmarks: 
        lh = np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten()
    else :
        lh = np.zeros(21*3)
    if results.right_hand_landmarks: 
        rh = np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten()
    else :
        rh = np.zeros(21*3)
    if results.face_landmarks: 
        face = np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten()
    else :
        face = np.zeros(1404)
    return np.concatenate([pose,face,lh,rh])

In [58]:
extract_keypoints(results).shape

NameError: name 'results' is not defined

In [59]:
DATA_PATH =os.path.join('TRAFFIC_SIGNALS_by_girish')
actions = np.array(['left','right','stop'])
no_sequences = 30
sequence_length = 30

In [9]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH,action,str(sequence)))
        except:
            pass

In [9]:

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False
                # Make Detections
                results = holistic.process(image)
                # print(results.face_landmarks)

                # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
                image.flags.writeable = True
                # Recolor image back to BGR for rendering
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                # 1. Draw face landmarks
                mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic. FACEMESH_TESSELATION, 
                                         mp_drawing.DrawingSpec(color=(0,255,255), thickness=1, circle_radius=1),
                                         mp_drawing.DrawingSpec(color=(0,165,255), thickness=1, circle_radius=1)
                                         )

                # 2. Right hand
                mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                         mp_drawing.DrawingSpec(color=(0,255,255), thickness=2, circle_radius=4),
                                         mp_drawing.DrawingSpec(color=(47,255,174), thickness=2, circle_radius=2)
                                         )

                # 3. Left Hand
                mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                         mp_drawing.DrawingSpec(color=(0,255,255), thickness=2, circle_radius=4),
                                         mp_drawing.DrawingSpec(color=(47,255,174), thickness=2, circle_radius=2)
                                         )

                # 4. Pose Detections
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                         mp_drawing.DrawingSpec(color=(0,255,255), thickness=2, circle_radius=4),
                                         mp_drawing.DrawingSpec(color=(47,255,174), thickness=2, circle_radius=2)
                                         )
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [60]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [61]:
import sklearn
import tensorflow as tf

In [62]:
import sklearn
import tensorflow as tf

In [63]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [64]:
label_map = {label:num for num, label in enumerate(actions)}

In [65]:
label_map

{'left': 0, 'right': 1, 'stop': 2}

In [66]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
            sequences.append(window)
            labels.append(label_map[action])

In [67]:
np.array(sequences).shape

(2700, 30, 1662)

In [68]:

y = to_categorical(labels).astype(int)

In [69]:
X = np.array(sequences)

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [71]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping,ReduceLROnPlateau

In [72]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)


In [73]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [74]:
res = [.7, 0.2, 0.1]

In [75]:
actions[np.argmax(res)]

'left'

In [76]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [77]:
checkpoint = ModelCheckpoint(r"C:\Users\analy\Downloads\weights\bestweight_final_girish.h5",
                             monitor="loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor = 'loss',
                              factor = 0.2,
                              patience = 3,
                              verbose = 1,
                              min_delta = 0.00001)



In [78]:
model.fit(X_train, y_train, epochs=1000, callbacks=[checkpoint])

Epoch 1/1000


InternalError:  Blas GEMM launch failed : a.shape=(32, 64), b.shape=(64, 256), m=32, n=256, k=64
	 [[{{node sequential_1/lstm_3/while/body/_1/sequential_1/lstm_3/while/lstm_cell_3/MatMul_1}}]] [Op:__inference_train_function_5780]

Function call stack:
train_function


In [62]:
res = model.predict(X_test)

In [63]:
model.save('bestweight.h5')

In [64]:
model.load_weights('bestweight.h5')

In [65]:

from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [66]:
yhat = model.predict(X_test)

In [67]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()


In [68]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[3, 0],
        [0, 2]],

       [[4, 0],
        [0, 1]],

       [[3, 0],
        [0, 2]]], dtype=int64)

In [69]:

accuracy_score(ytrue, yhat)

1.0

In [70]:
sequence =[]
threshold = 0.4

cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        image.flags.writeable = True
        # Recolor image back to BGR for rendering
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(0,255,255), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(0,165,255), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(0,255,255), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(47,255,174), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(0,255,255), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(47,255,174), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(0,255,255), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(47,255,174), thickness=2, circle_radius=2)
                                 )
        keypoints = extract_keypoints(results)
        sequence.insert(0,keypoints)
        sequence = sequence[:30]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence,axis=0))[0]
        cv2.rectangle(image,(0,0),(100,100),(47,255,174),-1)
        cv2.putText(image, actions[np.argmax(res)], (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

TypeError: only size-1 arrays can be converted to Python scalars

<Figure size 1296x1296 with 0 Axes>